In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
%matplotlib inline
import os
from scipy.stats import gaussian_kde
import metpy.calc as mpcalc
from metpy.units import units

In [4]:
if os.environ['COMPUTERNAME'] == 'DESKTOP-EQAO3M5':
  computer_flag = 'home'
else:
  computer_flag = 'office'

if computer_flag == 'home':
  file_dir = "F:/github/pythonScript/seafog/"
else:
  file_dir = "H:/github/python/seafog/"

def randrange(n, vmin, vmax):
    """
    Helper function to make an array of random numbers having shape (n, )
    with each number distributed Uniform(vmin, vmax).
    """
    return (vmax - vmin)*np.random.rand(n) + vmin

def linear_vis(x):
    if(x <= 1000.0):
      y = x/1000.0
    elif(x > 1000.0 and x <= 10000.0):
      y = (x-1000.0)/9000.0 + 1.0
    elif( x > 10000.0  and x <= 30000.0):
      y = (x-10000.0)/20000 + 2.0
    else:
      y = 3.0
    return y

def reverse_linear_vis(x):
    if(x<0):
      y = 1
    elif(x <= 1.0):
      y = x*1000.0
    elif(x <= 2.0):
      y = (x - 1.0)*9000.0 + 1000.0
    elif(x <= 3.0):
      y = (x-2.0)*20000 + 10000.0
    else:
      y = 30000.0
    return y

In [5]:
file_59754 = os.path.normpath(os.path.join(file_dir, './data/station/59754.by_5min_20200101-20221029.csv')) # 读取徐闻站分钟级数据
df_59754 = pd.read_csv(file_59754,sep=',',na_values=[9999])
df_59754.index = pd.to_datetime(df_59754["DDATETIME"])

df_code = pd.read_csv(os.path.normpath(os.path.join(file_dir, './code_readme.csv')),sep=',')

C:\Users\henry_Hill\AppData\Local\Temp\ipykernel_11548\673614758.py:2: DtypeWarning: Columns (12,112) have mixed types. Specify dtype option on import or set low_memory=False.
  df_59754 = pd.read_csv(file_59754,sep=',',na_values=[9999])


In [7]:
bjtime_index_59754 = df_59754.index.to_series() + pd.Timedelta(8,unit='h')

In [6]:
code_info = {
  'vis01':'V20059',
  'vis_01min':'V20001_701_01',
  'vis_10min':'V20001_701_10',
  'vis':'V20001',
  'td': 'V12003',
  't2m': 'V12001',
  "year":"V04001",
  "month": "V04002",
  "day": "V04003",
  "hour":"V04004",
  "minute":"V04005",
  "rhum":"V13003",
  'rain01':'V13019',
}

In [8]:
ds_G7425 = pd.read_hdf(os.path.normpath(os.path.join(file_dir, './data/G7425/G7425_202012to202209.hdf')), mode='r') # 浮标站数据

FileNotFoundError: File H:\github\python\seafog\data\G7425\G7425_202012to202209.hdf does not exist

In [15]:
df_start_end = pd.read_csv(os.path.normpath(os.path.join(file_dir, './src/statistics/分钟级-开始结束时间.csv')))
df_start_end['start'] = pd.to_datetime(df_start_end['start'], format='%Y/%m/%d  %H:%M')
df_start_end['end'] = pd.to_datetime(df_start_end['end'], format='%Y/%m/%d  %H:%M')
print(df_start_end)

                 start                 end
0  2020-12-27 18:30:00 2020-12-27 23:55:00
1  2021-01-15 16:40:00 2021-01-15 19:25:00
2  2021-01-20 16:10:00 2021-01-20 16:35:00
3  2021-01-22 04:20:00 2021-01-22 05:25:00
4  2021-01-22 12:55:00 2021-01-23 04:35:00
5  2021-01-23 12:45:00 2021-01-25 02:05:00
6  2021-01-25 14:00:00 2021-01-26 00:10:00
7  2021-01-26 09:10:00 2021-01-27 00:45:00
8  2021-02-02 13:10:00 2021-02-02 23:30:00
9  2021-02-05 17:25:00 2021-02-05 22:50:00
10 2021-02-06 21:45:00 2021-02-06 23:20:00
11 2021-02-08 18:10:00 2021-02-08 19:35:00
12 2021-02-09 00:35:00 2021-02-09 19:05:00
13 2021-02-11 20:55:00 2021-02-12 00:55:00
14 2021-02-12 16:35:00 2021-02-13 00:45:00
15 2021-02-14 14:00:00 2021-02-15 00:25:00
16 2021-02-15 23:15:00 2021-02-16 00:00:00
17 2021-02-16 16:05:00 2021-02-17 01:25:00
18 2021-02-20 21:50:00 2021-02-21 00:05:00
19 2021-02-21 20:50:00 2021-02-22 00:05:00
20 2021-02-22 18:05:00 2021-02-23 00:10:00
21 2021-02-23 20:15:00 2021-02-24 00:05:00
22 2021-02-

In [14]:

print(df_start_end['start'])

0    2020-12-27 18:30:00
1    2021-01-15 16:40:00
2    2021-01-20 16:10:00
3    2021-01-22 04:20:00
4    2021-01-22 12:55:00
5    2021-01-23 12:45:00
6    2021-01-25 14:00:00
7    2021-01-26 09:10:00
8    2021-02-02 13:10:00
9    2021-02-05 17:25:00
10   2021-02-06 21:45:00
11   2021-02-08 18:10:00
12   2021-02-09 00:35:00
13   2021-02-11 20:55:00
14   2021-02-12 16:35:00
15   2021-02-14 14:00:00
16   2021-02-15 23:15:00
17   2021-02-16 16:05:00
18   2021-02-20 21:50:00
19   2021-02-21 20:50:00
20   2021-02-22 18:05:00
21   2021-02-23 20:15:00
22   2021-02-25 23:00:00
23   2021-03-28 21:50:00
24   2021-04-03 22:15:00
25   2021-11-21 19:50:00
26   2022-01-05 19:35:00
27   2022-01-31 23:50:00
28   2022-02-09 16:25:00
29   2022-02-27 17:30:00
30   2022-03-01 17:25:00
31   2022-03-02 00:20:00
32   2022-03-13 18:50:00
33   2022-03-15 00:00:00
34   2022-03-16 20:35:00
35   2022-04-21 21:45:00
Name: start, dtype: datetime64[ns]
